In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`

import $ivy.$                                  

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder().master("local[1]").getOrCreate()

import spark.implicits._
import org.apache.spark.sql.functions._

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql.SparkSession


spark: SparkSession = org.apache.spark.sql.SparkSession@246972bd
import spark.implicits._

import org.apache.spark.sql.functions._

In [3]:
val exemplo = spark.read.textFile("data/exemplo")

exemplo: org.apache.spark.sql.Dataset[String] = [value: string]

In [4]:
val splitHttpLine = udf((line: String) => {
    def bytesToLong(s: String) = if(s=="-") "0" else s
    val pattern = "(.+) - - \\[(.+)\\] \"(\\w+) (.+) (\\w+/\\d+[.]\\d+)\" ([0-9]+) (-|[0-9]+)".r
    val pattern(host, date, method, path, protocol, code, bytes) = line
    Array(host, date, method, path, protocol, code, bytesToLong(bytes))
})

splitHttpLine: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(
  <function1>,
  ArrayType(StringType, true),
  Some(List(StringType))
)

In [5]:
val reqSplitted = exemplo.select(splitHttpLine(col("value")).as("req"))

reqSplitted: org.apache.spark.sql.package.DataFrame = [req: array<string>]

In [8]:
val reqStruct = reqSplitted.select(col("req")(0).as("host"), col("req")(1).as("date"), col("req")(2).as("method"), col("req")(3).as("path"), col("req")(4).as("protocol"), col("req")(5).as("code"), col("req")(6).as("bytes"))

reqStruct: org.apache.spark.sql.package.DataFrame = [host: string, date: string ... 5 more fields]

In [16]:
val uniqueHostsCount = reqStruct.groupBy(col("host")).count.collect.map(row=>row.getAs[String]("host")).size

unique_hosts: Int = 4

In [17]:
val codes404Count = reqStruct.where(col("code")===404).count

codes404Count: Long = 1L

In [30]:
val top5Hosts404 = reqStruct.where(col("code")===404).groupBy("host").count.sort(col("count").desc).collect.take(5).map(row=>row.getAs[String]("host"))

top5Hosts404: Array[String] = Array("js002.cc.utsunomiya-u.ac.jp")

In [31]:
val amount404PerDay = reqStruct.where(col("code")===404).select(col("date"))

amount404PerDay: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [host: string, date: string ... 5 more fields]

In [22]:
val totalBytesReturned = reqStruct.select(sum("bytes").as("bytes")).collect.head.getAs[Double]("bytes").toLong

totalBytesReturned: Long = 3552L

In [ ]:
val isoFormat = udf((line: String) => {
    def bytesToLong(s: String) = if(s=="-") "0" else s
    val pattern = "(.+) - - \\[(.+)\\] \"(\\w+) (.+) (\\w+/\\d+[.]\\d+)\" ([0-9]+) (-|[0-9]+)".r
    val pattern(host, date, method, path, protocol, code, bytes) = line
    Array(host, date, method, path, protocol, code, bytesToLong(bytes))
})